In [ ]:
import os
import numpy as np
import tensorflow as tf
import shutil
from sklearn.model_selection import LeaveOneOut
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# TensorFlow 로그 레벨 설정
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# GPU 설정 및 라이브러리 로드 확인
def configure_gpu():
    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print("GPU 설정 완료")
        else:
            print("GPU를 사용할 수 없습니다. TensorFlow 설치 및 환경을 확인하세요.")
    except Exception as e:
        print(f"GPU 설정 중 오류 발생: {e}")

configure_gpu()

In [ ]:
# 데이터 경로 및 저장 디렉토리
base_dir = "/home/bcml1/sigenv/eeg_band_split"
train_dir = os.path.join(base_dir, "train_data")
test_dir = os.path.join(base_dir, "test_data")
model_dir = "/home/bcml1/sigenv"  # 모델 저장 디렉토리

# 디렉토리 생성
def create_dirs():
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    os.makedirs(model_dir, exist_ok=True)

In [ ]:
# 데이터 및 라벨 로드 및 분리 함수
def split_samples(data_dir):
    file_names = sorted([f for f in os.listdir(data_dir) if f.endswith("_bands.npy")])  # npy 파일만 필터링
    loo = LeaveOneOut()

    for test_idx, train_idx in loo.split(file_names):
        test_sample = file_names[test_idx[0]]
        train_samples = [file_names[i] for i in train_idx]

        # 테스트 데이터 복사
        test_file_path = os.path.join(data_dir, test_sample)
        test_dest_path = os.path.join(test_dir, test_sample)
        if os.path.exists(test_file_path):
            shutil.copy(test_file_path, test_dest_path)
        else:
            print(f"Error: Test file {test_file_path} not found.")
            continue

        # 훈련 데이터 복사
        for train_sample in train_samples:
            train_file_path = os.path.join(data_dir, train_sample)
            train_dest_path = os.path.join(train_dir, train_sample)
            if os.path.exists(train_file_path):
                shutil.copy(train_file_path, train_dest_path)
            else:
                print(f"Error: Train file {train_file_path} not found.")

        yield train_samples, test_sample

In [ ]:
# Conv1D 기반 모델 정의
def build_cnn_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(32, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Conv1D(64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Conv1D(128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.4),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
# Main Process
create_dirs()

# 데이터 및 레이블 준비
all_X = []
all_y = []
file_names = sorted(os.listdir(base_dir))

for file_name in file_names:
    if file_name.endswith(".npy"):
        file_path = os.path.join(base_dir, file_name)
        band_data = np.load(file_path, allow_pickle=True)
        alpha_band = band_data[1]  # Alpha band 선택

        label = 1 if "positive" in file_name else 0
        for ch_idx in range(alpha_band.shape[1]):
            all_X.append(alpha_band[:, ch_idx])
            all_y.append(label)

In [ ]:
all_X = np.array(all_X)
all_y = np.array(all_y)
all_X = all_X / (np.max(np.abs(all_X), axis=1, keepdims=True) + 1e-8)
all_X = np.expand_dims(all_X, axis=-1)

In [ ]:
# 모델 생성
input_shape = all_X.shape[1:]
model = build_cnn_model(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# 모델 구조 디버깅
print("Model Summary:")
model.summary()

# LOOCV로 학습
loo = LeaveOneOut()
iteration = 1

# tqdm 추가: 크로스바 표시 및 진행 상황 시각화
for train_idx, test_idx in tqdm(loo.split(all_X), total=len(all_X), desc="Training Progress"):
    print(f"LOOCV Iteration {iteration}")

    X_train, X_test = all_X[train_idx], all_X[test_idx]
    y_train, y_test = all_y[train_idx], all_y[test_idx]

    # 데이터 디버깅
    print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

    # 학습 과정에서 verbose=1로 에포크 출력 추가
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

    print(f"Iteration {iteration} - Test Loss: {loss}, Test Accuracy: {accuracy}")
    iteration += 1

In [ ]:
# 모델 저장
model_save_path = os.path.join(model_dir, "test_eeg_cnn_model.h5")
model.save(model_save_path)
print(f"testing model saved at {model_save_path}")